In [1]:
darks = io.l1a_filenames('-?uvdark')

In [1]:
from IPython.parallel import Client
c = Client()

In [2]:
dview = c.direct_view()
lview = c.load_balanced_view()

In [3]:
def process_fname(fname):
    import numpy as np
    l1a = io.L1AReader(fname)
    header_df = io.get_header_df(l1a.hdulist[0]).T.convert_objects()
    if not hasattr(header_df, 'NAXIS3'):
        header_df['NAXIS3'] = np.nan
    header_df.drop(['XUV', 'OBS_ID', 'INT_TIME'], axis=1, inplace=True)
    df = pd.concat([header_df,
                    l1a.Engineering.T.convert_objects(),],
                   axis=1)
    df['dark_mean'] = l1a.img.mean()
    if df.size != 61:
        return (df, False)
    else:
        savepath = io.save_to_hdf(df, fname, 'l1a_dark_scans')
        return (df, True)

In [5]:
results = lview.map_async(process_fname, darks)

In [27]:
for res in results:
    print(res)

In [9]:
from iuvs import multitools

In [10]:
darkfnames = io.l1a_filenames('-?uvdark', iterator=False)
multitools.progress_display(results, darkfnames, sleep=20)

45.0 % done.
47.3 % done.
49.9 % done.
52.1 % done.
54.7 % done.
57.2 % done.
59.6 % done.
62.4 % done.
64.9 % done.
67.3 % done.
69.4 % done.
72.4 % done.
74.9 % done.
77.6 % done.
80.8 % done.
83.3 % done.
85.8 % done.
88.9 % done.
91.7 % done.
94.3 % done.
97.1 % done.


In [1]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [2]:
len(h5fnames)

9807

In [3]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [4]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48
Chunk 49


In [5]:
superdf = pd.concat(dfs, ignore_index=True)

In [6]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807 entries, 0 to 9806
Data columns (total 63 columns):
AVERAGE               9807 non-null float64
BIN_SHIFT             9807 non-null int64
BIN_SHIFT_DIR         9807 non-null int64
BIN_TBL               9807 non-null object
BIN_TYPE              9807 non-null object
BIN_X_ROW             9807 non-null int64
BIN_Y_ROW             9807 non-null int64
BITPIX                9807 non-null int64
BLANK                 9807 non-null int64
BSCALE                38 non-null float64
BZERO                 38 non-null float64
CADENCE               9807 non-null float64
CAPTURE               9807 non-null object
CASE_TEMP             9769 non-null float64
CHECKSUM              9807 non-null float64
DATA_COMPRESSION      9807 non-null int64
DET_TEMP              9769 non-null float64
EXTEND                9807 non-null bool
FILENAME              9807 non-null object
FUV_BAD_PIXEL_MASK    9807 non-null int64
IMAGE_NUMBER          9807 non-null floa

In [7]:
superdf.to_hdf('/home/klay6683/output/l1a_dark_stats.h5','df')

In [8]:
!ls -la /home/klay6683/output

total 247188
drwx--x--x  3 klay6683 wheel   29508 Apr  1 03:04 .
drwx--x--x 57 klay6683 wheel     111 Apr  1 02:58 ..
drwx--x--x  2 klay6683 wheel    9809 Apr  1 02:49 l1a_dark_scans
-rw-------  1 klay6683 wheel 7275512 Apr  1 03:04 l1a_dark_stats.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 02:37 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T090232_v01_r01.fits.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 01:44 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T090522_v01_r01.fits.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 01:43 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T091335_v01_r01.fits.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 01:44 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T091856_v01_r01.fits.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 00:57 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T092438_v01_r01.fits.h5
-rw-------  1 klay6683 wheel   85272 Feb  6 00:46 mvn_iuv_l1b_apoapse-orbit00107-mode2001-fuv_20141018T093128_v01_r01.f